# API (Application Programming interface)
애플리케이션과 프로그래밍으로 소통하는 방법. 서로 다른 프로그램에서 요청과 응답을 받을 수 있도록 만든 체계

## Web API
웹 서버/웹 브라우저를 위한 API. 현대 웹 개발은 여러 open API들을 활용.
youtube API, Google Map API, Naver Papago API, Kakao Map API 등등

### REST
API server를 개발하기 위한 일종의 소프트웨어 설계 방법론

--> REST API : REST라는 설계 디자인 약속을 지켜 구현한 API

**RESR에서 자원을 정의하고 주소를 지정하는 방법**
1. 자원의 식별 - URI
2. 자원의 행위 - HTTP method
3. 자원의 표현 - JSON/ 궁극적으로 표현되는 데이터 결과물



### REST-자원의 식별
**URI**
uniform resource identifire 통합 자원 식별자.
인터넷에서 리소스(자원)을 식별하는 문자열.
가장 일반적인 URI는 웹 주소로 알려진 URL

**URL** uniform resource locator 통합 자원 위치.
웹에서 주어진 리소스의 주소. 네트워크 상에 리소스가 어디 있는지를 알려주기 위한 약속

![image.png](attachment:image.png)

* Schema(or Protocol)
- 브라우저가 리소스를 요청하는 데 사용해야 하는 규약.
- URL의 첫 부분은 브라우저가 어떤 규약을 사용하는지를 표현
- 기본적으로 웹은 HTTP(S)를 요구(S:secure). 메일을 열기 위한 mailto:, 파일을 전송하기 위한 ftp:등 다른 프로토콜도 존재

* Domain Name
- 요정 중인 웹 서버. 어떤 웹 서버가 요구되는지를 가리킴. 직접 IP주소를 사용할수도 있지만 사람이 외우기 어렵기 때문에 주로 domain name으로 사용. 

* Port
- 웹 서버의 리소스에 접근하는 데 사용되는 기술적인 문(gate).
- HTTP 프로토콜의 표준 포트 : HTTP-80 (8000-> 80번 포트의 00번 게이트) HTTPS-443
- 표준 포트만 생략 가능함.

* Path
- 웹 서버의 리소스 경로. 초기에는 실제 파일이 위치한 물리적 위치를 나타냈지만, 오늘날은 실제 위치가 아닌 추상화된 형태의 구조를 표현. 

* Parameters
- 웹 서버에 제공하는 추가적인 데이터. & 기호로 구분되는 key-value쌍 목록. 서버는 리소스를 응답하기 전에 이러한 파라미터를 사용해 추가 작업을 수행할 수 있음

* Anchor
- 일종의 북마크. 브라우저에 해당 지점에 있는 콘텐츠 표시.
- fragment identifire(부분 식별자)라고 부르는 # 이후 부분은 서버에 전송되지 않음.



### REST-자원의 행위

**HTTP Request Methods
리소스에 대한 행위(수행하고자 하는 동작)를 정의. HTTP vervs라고도 함

대표  HTTP Request Methods
1. GET - 서버에 리소스의 표현을 요청. GET을 사용하는 요청은 데이터만 검색해야 함
2. POST - 데이터를 지정된 리소스에 제출. 서버의 상태를 변경
3. PUT - 요청한 주소의 리소스를 수정
4. DELETE - 지정된 리소스를 삭제

**HTTP response status codes**
특정 HTTP 요청이 성공적으로 완료되었는지 여부
100~599까지 5개의 응답 그룹


### REST-자원의 표현

서버가 응답할 수 있는 것은 페이지 뿐만 아니라 다양한 데이터 타입을 응답할 수 있다.
REST API는 이 중에서도 JSON 타입 응답 권장

html만을 응답하는 서버- render함수로 페이지 응답. 하지만 이 경우 페이지가 매번 새로고침 됨

--> JSON데이터를 응답하는 REST API 서버로 변환

--> Django는 더이상 Template 부분에 대한 역할을 담당하지 않게 되며(풀스택x) 

프론트와 백엔드가 분리되어 구성 됨-- Django를 사용해 RESTful API서버를 구축

## Django REST framwork
Django에서 Restful API 서버를 쉽게 구축할 수 있도록 도와주는 오픈소스 라이브러리

**Serialization**
직렬화.
여러 시스템에서 활용하기 위해 데이터 구조나 객체 상태를 나중에 재구성할 수 있는 포맷으로 변환하는 과정



POSTMAN 설치
API 개발 도구

api decorator 꼭 사용


In [ ]:
# serializers.py

from rest_framework import serializers
from .models import Article


class ArticleListSerializer(serializers.ModelSerializer):
    class Meta:
        model = Article
        fields = ('id', 'title', 'content',)


class ArticleSerializer(serializers.ModelSerializer):
    class Meta:
        model = Article
        fields = '__all__'


In [ ]:
# views.py
from rest_framework.response import Response
from rest_framework.decorators import api_view
from rest_framework import status
from .models import Article
from .serializers import ArticleListSerializer, ArticleSerializer


@api_view(['GET', 'POST'])
def article_list(request):
    if request.method == 'GET':
        articles = Article.objects.all()
        serializer = ArticleListSerializer(articles, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = ArticleSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


@api_view(['GET', 'DELETE', 'PUT'])
def article_detail(request, article_pk):
    article = Article.objects.get(pk=article_pk)

    if request.method == 'GET':
        serializer = ArticleSerializer(article)
        return Response(serializer.data)

    elif request.method == 'DELETE':
        article.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)
    
    elif request.method == 'PUT':
        serializer = ArticleSerializer(article, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


++ 분명 함수 작성 잘 했는데 안 되었던 이유
: 마지막에 슬래시 안넣었더니 GET 이외의 동작들이 정상 실행되지 않았음
 왜인진 모르겠다.